In [1]:
import numpy as np
import pandas as pd
import math
import random
import mcts
import matplotlib.pyplot as plt
import json
from itertools import product
import os

from src.Env.Grid.Cont_Grid import Continuous_Grid
from src.Planners.H_MCTS_continuous.Plain_MCTS_cont import Plain_MCTS_Cont

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
l1_rows = 8
l1_cols = 8
l1_width = 2
l1_height = 2
grid_setting = (l1_rows, l1_cols, l1_width, l1_height)
num_barrier = 15

# Save Parameter Search Results

## parameters to look for
- explorationConstant
- iter_Limit
- alpha
- constant_c
- gamma

In [ ]:
def cumul_plot(iter_Limit, mcts_result, path):
    iteration_plot = np.zeros(iter_Limit+1)
    for iter in mcts_result["iter_cnt"].values():
        iteration_plot[iter-1]+=1

    def cumulative_count(iterations):
        x = np.arange(1, len(iterations))
        y = np.cumsum(np.array(iterations)[:-1])
        
        return x, y

    x, y = cumulative_count(iteration_plot)

    plt.plot(x, y)
    plt.xlabel('Iteration')
    plt.ylabel('Success Rate')
    plt.title('Success Rate vs Iteration')

    plt.xlim(1, iter_Limit)  # Set the x-axis limits
    plt.xticks(range(0, iter_Limit+1, 1000))  # Set the x-axis tick marks

    plt.savefig("{}/success_rate.png".format(path))
    plt.close()

In [ ]:
def test_plain_mcts(param, folder_name):
    with open("{}/result.json".format(folder_name), 'w') as result_file:
        mcts_result = {}
        mcts_result["iter_cnt"] = {}
        mcts_success_rate = 0

        explorationConstant = param[0]
        iter_Limit = param[1]
        alpha = param[2]
        constant_c = param[3]
        gamma = param[4]

        for random_seed in range(100):
            MCTS = Plain_MCTS_Cont(grid_setting, random_seed=random_seed,
                                num_barrier = num_barrier,
                                explorationConstant=1 / math.sqrt(explorationConstant),
                                alpha=alpha,
                                constant_c=constant_c,
                                gamma=gamma,
                                iter_Limit = iter_Limit)
            traj, success, iter = MCTS.search("{}/tree/{}.png".format(folder_name, random_seed))
            iter += 1
            mcts_result["iter_cnt"][random_seed] = iter

            if success:
                mcts_success_rate += 1
                MCTS.env.plot_grid(0, traj, "{}/traj/{}.png".format(folder_name, random_seed))
                print(f'success with {iter}')
            else:
                MCTS.env.plot_grid(0, traj, "{}/traj/{}.png".format(folder_name, random_seed))
                print('Failed')
        
        mcts_result["success_rate"] = mcts_success_rate
        json.dump(mcts_result, result_file)
        cumul_plot(iter_Limit, mcts_result, folder_name)
    
    

In [ ]:
def make_param_dir(path):
    # check whether directory already exists
    if not os.path.exists(path):
        os.mkdir(path)
        print("Folder %s created!" % path)
    else:
        print("Folder %s already exists" % path)

In [ ]:
# explorationConstant = np.linspace(1.5, 3, 4)
# iter_Limit = [10000]
# alpha = [0.05]
# constant_c = [2, 4, 8, 16]
# gamma = [1]

explorationConstant = [1.0, 1.5, 2.0, 2.5, 3.0]
iter_Limit = [10000]
alpha = [0.15, 0.2]
constant_c = [4, 8, 16]
gamma = [1]

# Create a list of parameter arrays
parameters = [explorationConstant, iter_Limit, alpha, constant_c, gamma]

# Generate all possible combinations
param_combinations = list(product(*parameters))

# Print the combinations
for param in param_combinations:
    print("Param to check", param)

    folder_name = "./experiments/Plain-MCTS-wTree/plain_mcts_{}_{}_{}_{}_{}" \
                .format(param[0], param[1], param[2], param[3], param[4])
    make_param_dir(folder_name)
    make_param_dir("{}/traj".format(folder_name))
    make_param_dir("{}/tree".format(folder_name))
    test_plain_mcts(param, folder_name)

Param to check (1.0, 10000, 0.05, 2, 1)
Folder ./experiments/Plain-MCTS-wTree/plain_mcts_1.0_10000_0.05_2_1 already exists
Folder ./experiments/Plain-MCTS-wTree/plain_mcts_1.0_10000_0.05_2_1/traj already exists
Folder ./experiments/Plain-MCTS-wTree/plain_mcts_1.0_10000_0.05_2_1/tree already exists
success with 60
Failed
Failed
success with 154
success with 98
success with 117
Failed
Failed
Failed
Failed
success with 142
success with 183
success with 59
success with 474
success with 242
success with 25
Failed
success with 264
success with 36
success with 72
Failed
success with 795
Failed
Failed
success with 45
Failed
success with 270
success with 266
Failed
success with 75
Failed
Failed
Failed
success with 112
success with 46
success with 202
success with 141
success with 229
success with 59
success with 161
success with 170
success with 8665
success with 137
success with 135
Failed
success with 363
success with 223
success with 355
success with 143
success with 376
Failed
success with 

: 

: 